# Step19, 변수 사용성 개선

## 19.1 변수 이름 지정 

수많은 변수를 처리할 것이므로 변수들을 서로 구분할 필요가 있다. --> 변수에 '이름'을 붙여준다.  
Variable 클래스에 name이라는 인스턴스 변수를 추가한다.

In [1]:
class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):                            # 입력받는 데이터가 ndarray 구조가 아니면 오류 발생
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data        # 데이터 선언
        self.name = name
        self.grad = None        # 미분값 선언
        self.creator = None     # 이 데이터가 어디출신인지, 어느 공장에서 만들어졌는지 표기
        self.generation = 0     # 세대를 기록하는 변수

    def set_creator(self, func):    # 생성자 = 공장 = 함수
        self.creator = func 
        self.generation = func.generation + 1   # 세대를 기록한다(부모 세대 + 1)

    def cleargrad(self):
        self.grad = None                        # 미분값 초기화

    def backward(self, retain_grad=False):      # 추가되는곳 : 
        if self.grad is None:
            self.grad = np.ones_like(self.data)     # 미분값이 없으면 모두 1로 구성된 행렬

        funcs = []                      # <-- 바뀐부분
        seen_set = set()                # 집합은 중복을 막는다.

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)              # DezZero 함수 리스트를 세대순으로 정렬하는 역할 
                                            # 그결과 funcs.pop()은 자동으로 세대가 가장 큰 DeZero 함수 순으로 꺼낸다. 

        while funcs:
            f = funcs.pop()                         # 함수들을 하나씩 뽑는다.
            gys = [output().grad for output in f.outputs]     # 출력변수인 outputs에 담겨있는 미분값(.grad)들을 리스트에 담는다
            gxs = f.backward(*gys)                          # f의 역전파를 호출한다. *를 붙혀 리스트를 풀면서 넣어준다.(리스트 언팩)
            if not isinstance(gxs, tuple):                  # gxs가 튜플이 아니면 튜플로 변환한다.
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):                # gxs와 f.inputs의 각 원소는 서로 대응 관계
                if x.grad is None:
                    x.grad = gx                             # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장
                else:
                    x.grad = x.grad + gx    # x.grad가 None이 아니라 기존에 가지고 있는 값이 있다면 가지고 있는 값에 gx를 추가로 더한다.

                if x.creator is not None:
                    add_func(x.creator)      # <-- 바뀐부분, 수정전: funcs.append(x.creator) 출처가 있는 데이터를 add_funcs에 넣는다.
            
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None 

**NOTE_** 변수에 이름을 붙일 수 있다면 계산 그래프를 시각화할때 변수 이름을 그래프에 표시할 수 있다.

## 19.2 ndarray 인스턴스 변수 
Variable이 데이터인 것처럼 보이게 하는 장치를 만든다.  
Variable 클래스는 ndarray만 취급하기로 하였다.  
이제는 Variable 인스턴스 자체가 ndarray 인스터스처럼 보이게 한다.


ndarray의 shape 인스턴스, ndim, size, dtype 변수처럼 만들기 

~~~python
class Variable:
    # ... 생략 ...

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim
    
    @property 
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
~~~

@property 이것으로 shape 메서드를 인스턴스 변수처럼 사용할 수 있게된다.

메서드 호출이 아닌 인스턴스 변수로 데이터의 형상을 얻을 수 있다. 같은 방법으로 ndarray의 다른 인스턴스 변수들을 Variable에 추가할 수 있다.



## 19.3 len 함수와 print 함수 

이어서 Variable 클래스를 더 확장하여 파이썬의 len 함수와도 함께 사용할 수 있도록 한다

~~~python 
class Variable:
    # ... 생략 ...
    
    def __len__(self):
        return len(self.data)
~~~

이와 같이 __len__이라는 특수 메서드를 구현하면 Variable 인스턴스에 대해서도 len 함수를 사용할 수 있데 된다.

**NOTE_** 파이썬에서 __init__, __len__ 등 특별한 의미를 지닌 메서드는 밑줄 두개로 감싼 이름을 사용한다.

마지막으로 Variable의 내용을 쉽게 확인할 수 있는 기능을 추가한다.  
print 함수를 사용하여 Variable 안의 데이터 내용을 출력하는 기능이다.

~~~python 
class Variable:
    # ... 생략 ...

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'
~~~


최종 코드 

In [5]:
import numpy as np

class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):                            # 입력받는 데이터가 ndarray 구조가 아니면 오류 발생
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data        # 데이터 선언
        self.name = name
        self.grad = None        # 미분값 선언
        self.creator = None     # 이 데이터가 어디출신인지, 어느 공장에서 만들어졌는지 표기
        self.generation = 0     # 세대를 기록하는 변수
    
    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):    # 생성자 = 공장 = 함수
        self.creator = func 
        self.generation = func.generation + 1   # 세대를 기록한다(부모 세대 + 1)

    def cleargrad(self):
        self.grad = None                        # 미분값 초기화

    def backward(self, retain_grad=False):      # 추가되는곳 : 
        if self.grad is None:
            self.grad = np.ones_like(self.data)     # 미분값이 없으면 모두 1로 구성된 행렬

        funcs = []                      # <-- 바뀐부분
        seen_set = set()                # 집합은 중복을 막는다.

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)              # DezZero 함수 리스트를 세대순으로 정렬하는 역할 
                                            # 그결과 funcs.pop()은 자동으로 세대가 가장 큰 DeZero 함수 순으로 꺼낸다. 

        while funcs:
            f = funcs.pop()                         # 함수들을 하나씩 뽑는다.
            gys = [output().grad for output in f.outputs]     # 출력변수인 outputs에 담겨있는 미분값(.grad)들을 리스트에 담는다
            gxs = f.backward(*gys)                          # f의 역전파를 호출한다. *를 붙혀 리스트를 풀면서 넣어준다.(리스트 언팩)
            if not isinstance(gxs, tuple):                  # gxs가 튜플이 아니면 튜플로 변환한다.
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):                # gxs와 f.inputs의 각 원소는 서로 대응 관계
                if x.grad is None:
                    x.grad = gx                             # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장
                else:
                    x.grad = x.grad + gx    # x.grad가 None이 아니라 기존에 가지고 있는 값이 있다면 가지고 있는 값에 gx를 추가로 더한다.

                if x.creator is not None:
                    add_func(x.creator)      # <-- 바뀐부분, 수정전: funcs.append(x.creator) 출처가 있는 데이터를 add_funcs에 넣는다.
            
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None 

In [9]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
print(x.shape)
print(x.ndim)
print(x.size)
print(x.dtype)
print(len(x))
print(x)

(2, 3)
2
6
int32
2
variable([[1 2 3]
          [4 5 6]])
